In [26]:
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import load, Loader
import subprocess
from json import dump

## Parameters for Processing

In [27]:
output_path = Path("sample_output")
actual_path = output_path / "actual_time.yaml"
theoretical_path = output_path / "theoretical_time.yaml"
metadata_path = output_path / "metadata.yaml"
mem_folder_path = output_path / "memory_output"

data_dump_path = Path("processed_output")
output_fnames = ("actual_runtime.csv", "theoretical_runtime.csv", "bytes.csv", "all_fields.json")
MFLOPS = 870
FLOPS = MFLOPS * 10E6

## Loading Metadata

In [28]:
with open(metadata_path, "r") as f:
    metadata = load(f, Loader=Loader)
    
# metadata

## Loading Actual Times

In [29]:
with open(actual_path, "r") as f:
    actual_times = load(f, Loader=Loader)
    
actual_times_dict = {reg_name: {row_count: nanoseconds for row_count, nanoseconds in actual_times} for reg_name, actual_times in actual_times.items()}
# actual_times_dict

## Loading Theoretical Flop Counts

In [30]:
with open(theoretical_path, "r") as f:
    theoretical_flops = load(f, Loader=Loader)
    
theoretical_times_dict = {reg_name: {row_count: (flops / FLOPS) * 10E9 for row_count, flops in theoretical_flops} for reg_name, theoretical_flops in theoretical_flops.items()}
# theoretical_times_dict

## Loading / Processing Binary Memory Files

In [31]:
make_mempath = lambda reg_name, row_count, iter, ext: mem_folder_path / f"{'mem' if ext == 'bin' else 'memray-csv-mem'}_{reg_name}_{row_count}_{iter}.{ext}"

mem_dict = {}
for reg_name in metadata["reg_names"]:
    mem_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        runs = []
        for iter in range(metadata["repeat"]):
            # use memray transform to actually create csv
            input_path = make_mempath(reg_name, row_count, iter, "bin")
            print(f"{input_path =}")
            subprocess.run(["memray", "transform", "csv", input_path, "-f"])
            
            # load csv normally, sum all mem usage
            output_path = make_mempath(reg_name, row_count, iter, "csv")
            single_mem_df = pd.read_csv(output_path, header=0, index_col=None)
            bytes_used = single_mem_df["size"].sum()
            runs.append(bytes_used)
            
        mem_dict[reg_name][row_count] = np.mean(runs)

input_path =PosixPath('sample_output/memory_output/mem_tf-necd_10_0.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote sample_output/memory_output/memray-csv-mem_tf-necd_10_0.csv
input_path =PosixPath('sample_output/memory_output/mem_tf-necd_31_0.bin')

⚠  No symbol information was found for the Python interpreter  ⚠

Without symbolic information reports showing native traces may not accurately 
reflect stack traces. Please use an interpreter built with debug symbols for 
best results. Check https://bloomberg.github.io/memray/native_mode.html for more
information regarding how memray resolves symbols.


Wrote sample_output/memory_output/memray-csv-mem_tf-necd_31_0.csv
inpu

## Aggregate and Output Data

In [32]:
json_prep_dict = {}
for reg_name in metadata["reg_names"]:
    json_prep_dict[reg_name] = {}
    for row_count in metadata["rows_in_experiment"]:
        json_prep_dict[reg_name][row_count] = {
            "real_runtime": actual_times_dict[reg_name][row_count],
            "theoretical_runtime": theoretical_times_dict[reg_name][row_count],
            "bytes": mem_dict[reg_name][row_count]
        }

df_real_runtime = pd.DataFrame(actual_times_dict)
df_theoretical_runtime = pd.DataFrame(theoretical_times_dict)
df_memory = pd.DataFrame(mem_dict)

data_dump_path.mkdir(exist_ok=True)

df_real_runtime.to_csv(data_dump_path / output_fnames[0], header=True, index=True)
df_theoretical_runtime.to_csv(data_dump_path / output_fnames[1], header=True, index=True)
df_memory.to_csv(data_dump_path / output_fnames[2], header=True, index=True)

with open(data_dump_path / output_fnames[3], "w") as f:
    dump(json_prep_dict, f)
    
print(f"All data has been dumped to: {data_dump_path}")

All data has been dumped to: processed_output
